In [ ]:
import numpy as np
import os
import scipy.io
from scipy.fftpack import dct, idct
from scipy.optimize import minimize
import holoviews as hv; hv.extension("bokeh", logo=False)

hv.opts.defaults(hv.opts.Image( xaxis=None, yaxis=None,invert_yaxis=True),
                 hv.opts.Raster( xaxis=None, yaxis=None,invert_yaxis=True)) #cmap="gray", data_aspect=1

In [ ]:
def show_C_Psi_Theta( C, Psi, Theta ):
    h = hv.Raster( np.flipud(C) ).opts(title="C") +\
        hv.Raster( np.flipud(Psi) ).opts(title="Psi") +\
        hv.Raster( np.flipud(Theta) ).opts(title="Theta")
    return h.opts(shared_axes=False, title="Selection of Basis Vector Rows")

In [ ]:
mat = scipy.io.loadmat(os.path.join('..','DATA','CC2.mat'))
CC  = mat['CC']
#CC_map = ListedColormap(CC)

p = 14
n = 32

In [ ]:
Psi = dct(np.identity(n))
#hv.Image(np.flipud(Psi))

In [ ]:
C    = np.identity(n)
perm = np.random.permutation(n)[:p]
C    = C[perm,:] # compressed measurement

Theta = C @ Psi

#(hv.Image(1-np.flipud(C)).opts(cmap="gray")+hv.Image( np.flipud(Theta))).opts(shared_axes=False)
show_C_Psi_Theta( C, Psi, Theta )

In [ ]:
## Plot s, y
s     = np.zeros((n,1))
s[2]  = 1.4
s[13] = 0.7
s[27] = 2.2
y     = C @ Psi @ s

sL2        = np.linalg.pinv(Theta) @ y
sbackslash = np.linalg.lstsq(Theta,y,rcond=None)[0]

h = \
hv.Raster(np.flipud( s         )).opts(title="sparse s") +\
hv.Raster(np.flipud( sL2       )).opts(title='sL2')+\
hv.Raster(np.flipud( sbackslash)).opts(title='sbackslash')+\
hv.Raster(np.flipud( y         )).opts(title='reconstructed y')

h.opts(shared_axes=False).opts(hv.opts.Raster(width=100))

In [ ]:
## L1-Minimization using SciPy
def L1_norm(x):
    return np.linalg.norm(x,ord=1)

y      = y.reshape(-1)
constr = ({'type': 'eq', 'fun': lambda x:  Theta @ x - y})
x0     = np.linalg.pinv(Theta) @ y
res    = minimize(L1_norm, x0, method='SLSQP',constraints=constr)
s2     = res.x

In [ ]:
C     = np.random.randn(p,n)
Theta = C @ Psi

show_C_Psi_Theta( C, Psi, Theta ).opts(title="Random Combination of Basis Vector Rows")

In [ ]:
# Plot C and Theta (3) - Bernoulli Random
#C = np.random.randn(p,n)
C0 = C > 0

Theta = C0 @ Psi

show_C_Psi_Theta( C0, Psi, Theta ).opts(title="Random Combination of Basis Vector Rows C>0")

In [ ]:
# Plot C and Theta (4) - Sparse Bernoulli
#C = np.random.randn(p,n)
C1 = C > 1
Theta = C1 @ Psi

show_C_Psi_Theta( C1, Psi, Theta ).opts(title="Random Combination of Basis Vector Rows C>1")

In [ ]:
# Bad C and Theta (5) - DCT Meas
C = idct(np.identity(n))
perm = np.arange(n-p,n)
C = C[perm,:] # compressed measurement

Theta = C @ Psi

y = Theta @ s

show_C_Psi_Theta( C1, Psi, Theta ).opts(title="Random Combination of Basis Vector Rows: Bad C")